In [293]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn import SVD, accuracy
from surprise import SVD, accuracy, dump, prediction_algorithms

In [294]:
data = pd.read_csv('ratings.csv')

In [295]:
data = data[['userId', 'movieId', 'rating']]

In [296]:
data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [297]:
data.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [298]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

In [299]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.1)

In [300]:
# We'll use the SVD algorithm.
algo = SVD(n_factors=160, n_epochs=100, lr_all=0.005, reg_all=0.1)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mse(predictions)

MSE: 0.7093


0.7092729918324052

In [301]:
from surprise.model_selection import cross_validate
# Run 5-fold cross-validation and print results.
#cross_validate(algo, data, measures=['RMSE', 'MAE', 'MSE'], cv=10, verbose=True)

In [302]:
algo_svd = algo

In [303]:
# We'll use the SVD algorithm.
from surprise import SVDpp
algo = SVDpp(n_factors = 40, n_epochs= 20, lr_all= 0.02, reg_all= 0.08)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mse(predictions)

MSE: 0.7182


0.7181738159430315

In [304]:
algo_svdpp = algo

In [305]:
# We'll use the SVD algorithm.
algo = SVD(n_factors=800, n_epochs=100, lr_all=0.005, reg_all=0.1)
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mse(predictions)
algo_svd2 = algo

MSE: 0.7121


In [306]:
algos = [algo_svd, algo_svd2,  algo_svdpp]
preds = []
for algo in algos:
    preds += [algo.test(testset)]

In [307]:
#Делаем ансамбль моделей по принципу голосование 2 из 3
pr = []
d = 0.003
for pred in zip(preds[0],preds[1],preds[2]):
    est = 0
    for i in pred:
        est += i[3]
    est = est / len(pred)
    #est = pred[0][3]
    
    if (pred[0][3] - pred[1][3])  < d: est = pred[0][3]
    if (pred[0][3] - pred[2][3])  < d: est = pred[0][3]
    if (pred[1][3] - pred[2][3])  < d: est = pred[1][3]
    
    p = prediction_algorithms.predictions.Prediction(pred[0][0],pred[0][1],pred[0][2], est, pred[0][4])    
    pr += [p]

In [308]:
print(accuracy.mse(pr))

MSE: 0.7066
0.7065908825163597
